<a href="https://colab.research.google.com/github/YusakuH02/myRepository/blob/main/langchain_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install openai langchain llama-index google-search-results faiss-cpu

In [ ]:
!pip install langchain openai
!pip install faiss-cpu tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key="***your key***"
)

response = llm([HumanMessage(content="LangChainは動作していますか")])
print(response.content)


<ipython-input-4-614537a73967>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
<ipython-input-4-614537a73967>:9: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content="LangChainは動作していますか")])


LangChainは、現在動作していません。LangChainは、多言語翻訳や通訳を提供するプラットフォームでしたが、現在はサービスを停止しています。代わりに、他の翻訳サービスやプラットフォームを利用することをお勧めします。


In [ ]:
import requests

# URL定義（問題と解答）
url_202406_Q = "https://www.ipa.go.jp/shiken/mondai-kaiotu/m42obm000000afqx-att/2024r06a_ap_pm_qs.pdf"
url_202406_A = "https://www.ipa.go.jp/shiken/mondai-kaiotu/m42obm000000afqx-att/2024r06a_ap_pm_ans.pdf"

# 問題PDFのダウンロード
response_202406_Q = requests.get(url_202406_Q)
with open("202406_ap_pm_question.pdf", "wb") as f:
    f.write(response_202406_Q.content)

# 解答PDFのダウンロード
response_202406_A = requests.get(url_202406_A)
with open("202406_ap_pm_answer.pdf", "wb") as f:
    f.write(response_202406_A.content)

fitzでテキスト分割

In [ ]:
!pip install PyMuPDF

import fitz  # PyMuPDF
x = 0
def extract_text_from_pdf(x, pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for i, page in enumerate(doc):
        if i < x:
            continue  # 不要なページのスキップ
        text += page.get_text()
    return text

#pdf_text_202406_Q = extract_text_from_pdf(6, "202406_ap_pm_question.pdf")
pdf_text_202406_A = extract_text_from_pdf(0, "202406_ap_pm_answer.pdf")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 74.9 MB/s eta 0:00:00


OCRで画像認識して問題pdfをテキストに

In [ ]:
!apt install poppler-utils
!pip install pdf2image pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 0s (1,097 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.7_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.7) ...
Setting up poppler-utils (22.02.0-2ubuntu0.7) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!apt install tesseract-ocr-jpn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-jpn
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,390 kB of archives.
After this operation, 2,486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-jpn all 1:4.00~git30-7274cfa-1.1 [1,390 kB]
Fetched 1,390 kB in 1s (1,458 kB/s)
Selecting previously unselected package tesseract-ocr-jpn.
(Reading database ... 126363 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-jpn_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-jpn (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-jpn (1:4.00~git30-7274cfa-1.1) ...


In [ ]:
import pytesseract
from pdf2image import convert_from_path
import os

# OCR対象のPDFファイル
pdf_path = "202406_ap_pm_question.pdf"

# 画像に変換
images = convert_from_path(pdf_path)

# OCRでテキスト抽出
ocr_text = ""
for i, image in enumerate(images):
    text = pytesseract.image_to_string(image, lang="jpn")  # 日本語対応
    ocr_text += f"\n--- Page {i + 1} ---\n{text}"


In [ ]:
pdf_text_202406_Q = ocr_text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)

docs_202406_Q = text_splitter.create_documents([pdf_text_202406_Q])
docs_202406_A = text_splitter.create_documents([pdf_text_202406_A])

In [ ]:
print("1つ目のチャンク内容:")
#print(docs_202406_Q[0].page_content)
print(docs_202406_A[0].page_content)

1つ目のチャンク内容:
1/6 
©2024 独立行政法人情報処理推進機構 
令和６年度 秋期 応用情報技術者試験 解答例 
 
午後試験 
問１ 
出題趣旨 
 パスワードの管理方法が不適切であったことによって，不正アクセスの被害が拡大する事例が発生している。
 本問では，EC サイトに対するペネトレーションテストの結果を題材として，パスワードの安全な管理方法に
ついての理解を問う。 
 
設問 
解答例・解答の要点 
備考 
設問１ 
ハッシュ値からパスワードの割出しは難しい。 
 
設問２ 
機密性 
 
設問３ 
a
オ 
 
 
b
カ 
 
設問４ 
イ 
 
設問５ 
多層防御 
 
設問６(1) 会員テーブルの窃取だけではペッパーを得ることができないから 
 
 
(2)
c
704 
 
 
問２ 
出題趣旨 
 昨今，デジタル化，脱炭素化などによる人々の社会意識の変容が経営環境を変化させたことで，“ブランド”，
“人的資本経営”などの非財務情報に基づく戦略の策定が，経営課題としての重みを増してきている。 
 本問では，コーヒーチェーン店のブランド再構築による企業の成長や利益性向上を題材として，“人的資本”
を強化することで，顧客へのサービス品質向上につなげるサービスプロフィットチェーンに基づく戦略の策定
についての理解を問う。 
 
設問 
解答例・解答の要点 
備考 
設問１
(1)
a 
従業員満足度 
 
 
 
b 
顧客満足度 
 
 
(2)
機会 
環境や倫理に対する社会意識の高まり 
 
 
 
強み 
環境に配慮した高品質なコーヒー豆の使用 
 
 
(3)
c 
・顧客がQ 社にもたらす利益 
・顧客生涯価値 
 
 
 
(4)
d 
リピート率 
 
設問２
(1)
e 
自己評価と部下からの評価とのギャップ 
 
 
(2)
f 
共感 
 
 
(3) 香り深いコーヒーの淹れ方のマニュアル化 
 
 
2/6 
©2024 独立行政法人情報処理推進機構 
問３ 
出題趣旨 
 素数は，数学における学問的な用途のほか，暗号処理の分野などで活用されている。 
 本問では，素数を列挙するアルゴリズムの一つであるエラトステネスの篩
ふるい
を題材として，素数の性質を利用
した効率化の手法についての理解を問う。 
 
設問 
解

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key="***your key***"
)

In [ ]:
vectorstore_q = FAISS.from_documents(docs_202406_Q, embeddings)
vectorstore_a = FAISS.from_documents(docs_202406_A, embeddings)

In [ ]:
# 一定文字数以下 or 空白が多すぎるチャンクを除外
filtered_docs = [
    d for d in docs_202406_Q
    if len(d.page_content.strip()) > 200 and "問" in d.page_content
]

In [ ]:
for i, d in enumerate(docs_202406_Q[:10]):
    print(f"\n--- Chunk {i} ---\n")
    print(d.page_content[:800])


--- Chunk 0 ---

--- Page 1 ---
令和6年度 秋期
用情報技術者試験

午後 問題

試験時間     13:60 15:30 (2時間 36 分)

1. 試験開始及び終了は, 監督員の時計が基準です。監督貞の指示に従ってください。

注意事項

 

 

試験開始の合図があるまで, 問題冊子を開いて中を見てはいけません。
答案用紙への受験番号などの記入は, 試験開始の合図があってから始めてください

 

いこ】

問題は, 次の表に従って解答してください。

  
   

 

 

 

 

 

還証に:較

--- Chunk 1 ---

還証に:較

 
 
 

5. 答案用紙の記入に当たっては, 次の指示に従ってください。

 

 

 

 

(1) B 又は HB の黒鉛筆又はシャープペンシルを使
きい。

(2) 受験番号欄に受験番号を, 生年月日欄に受験票の生年月

してくだ

 

 

 

 

 

 

 

 

 

を記入してください。 正しく記入されていない場合

されないことがあります。 生年月日欄については,

 

 

 

 

 

 

 

 

生年月日を訂正した場合でも, 訂正前の生年月
ください。

--- Chunk 2 ---

生年月日を訂正した場合でも, 訂正前の生年月
ください。

 

 

 

 

 

 

(3) 選択した問題については, 右の例に従って, 選択欄の問題
番号を〇印で囲んでください。O〇印がない場合は,

 

 

 

 

 

ません。 問 2一問 11 について, 5 問以上〇印で書
はじめの 4 問について採点します。

(4) 解答は, 問題番号ごとに指定された枠内に記入
い。

(5) 解答は, 丁寧な字ではっきりと書いてください
い場合は, 減点の対象になります。

--- Chunk 3 ---

注意事項は問題冊子の裏表紙に続きます。
こちら側から裏返して, 必ず読んでください

 
 

は, 採点
受験票の
を記入して

採点され

んだ場合は

してくださ

。 読みにく

〔問3, 問4, 問6,
間 8 を選択 した場合
の例]

 

--- Page 2 ---

--- Page

In [ ]:
query = "プログラミング分野の問題を一題出題してください"
retrieved_question = vectorstore_q.similarity_search(query, k=1)[0].page_content

print("📝 問題：\n", retrieved_question)

📝 問題：
 プログラミング            素数を列挙するアルゴリズム

 

 
  

  

システムアーキテクチャ     データ処理機能の配置

 

 
 

セキュア Web ゲートウェイサービスの導入

 

 
 

トレーディングカードの個人間売買サイトの構築

 

 

 
 

組込みシステム開発        スマートイヤホン

 

 

                                                                                                                                                                                                                                                                 
                                                                                                                                                                                                                                                                 

情報システム開発          オブジェクト指向設計

 

 

 
  

   

 

剛    プロジェクトマネジメント |電気機器メーカーの新たなプロジェクト

 

 
 
 
  

サービスデスクの立上げ

システム監査

  

チャットボット導入における開発計画の監査

 

 

--- Page 4 ---
擬似言語の記述形式 (基本情報技術者試験, 応用情報技術者試験用)

 

 

擬似言語を使

 

 

 

 

 

 

 

 

 

 

 

 

 

 

した問題では, 各問題文中に注記がない限り, 次の記述形式が適

In [ ]:
user_answer = input("あなたの回答を入力してください：\n")

あなたの回答を入力してください：
あ


In [ ]:
retrieved_answer = vectorstore_a.similarity_search(retrieved_question, k=1)[0].page_content

feedback_prompt = f"""
以下は応用情報技術者試験の問題とその模範解答です。

【問題】
{retrieved_question}

【模範解答】
{retrieved_answer}

【ユーザーの回答】
{user_answer}

このユーザーの回答が適切かどうかを評価してください。不足があれば解説と修正案を提示してください。また、関連する知識やポイントがあれば追加で教えてください。
"""

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key="***your key***"
)
response = llm([HumanMessage(content=feedback_prompt)])

print("フィードバック：\n", response.content)


🧠 フィードバック：
 このユーザーの回答には不足があります。問題文には解答例や要点が記載されていますので、それらを参考に回答を修正する必要があります。

例えば、問題５の設問１において、NAPTとip1の間に関連性や説明が必要です。具体的にどのような関係があるのかを記述する必要があります。

また、他の設問についても同様に、解答例や要点を適切に組み込んで回答を修正する必要があります。

関連する知識やポイントとしては、セキュアWebゲートウェイサービスの適用検討において、NAPTやIPアドレスの設定などネットワークの変更設計や運用管理に関する知識が重要です。また、セキュリティ対策やネットワーク設計において、クラウドサービスの利用やセキュリティリスクの考慮も重要なポイントとなります。


↓問題の概要について答えるだけ

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa_chain = RetrievalQA.from_chain_type(
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key="***your key***"
),
    retriever=vectorstore.as_retriever(),
)

result = qa_chain.run("問3の内容を簡単に説明してください")

print("回答：")
print(result)


🧠 回答：
問3は、「ネットワーク構成の変更」に関する問題です。具体的には、IPアドレスや機器の名称を使って、特定の箇所に適切な字句を入れる問題です。この問題では、ネットワーク構成に関する知識や理解が問われます。
